In [1]:
pip install lightgbm

In [2]:
pip install catboost

In [3]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from lightgbm import LGBMClassifier
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import StratifiedKFold,KFold
# Restrict minor warnings
import warnings
from catboost import CatBoostClassifier, Pool
warnings.filterwarnings('ignore')
import seaborn as sns; sns.set()

In [4]:
train_x = pd.read_csv("train.csv")
test_x = pd.read_csv("test.csv")

In [5]:
train_x.nunique()

case_id                              318438
Hospital_code                            32
Hospital_type_code                        7
City_Code_Hospital                       11
Hospital_region_code                      3
Available Extra Rooms in Hospital        18
Department                                5
Ward_Type                                 6
Ward_Facility_Code                        6
Bed Grade                                 4
patientid                             92017
City_Code_Patient                        37
Type of Admission                         3
Severity of Illness                       3
Visitors with Patient                    28
Age                                      10
Admission_Deposit                      7300
Stay                                     11
dtype: int64

# Label Encoding the columns and mapping it into new values

In [6]:
cols = ['Hospital_type_code','City_Code_Hospital',
       'Hospital_region_code'
       ,'Department'
       ,'Ward_Type',
       'Ward_Facility_Code',
       'Bed Grade','Type of Admission','Severity of Illness','Age']

In [7]:
from sklearn.preprocessing import LabelEncoder
for i in cols:
    le = LabelEncoder()
    train_x[i] = le.fit_transform(train_x[i].astype('str'))
    le = LabelEncoder()
    test_x[i] = le.fit_transform(test_x[i].astype('str'))

In [8]:
le = LabelEncoder()
train_x['Stay'] =  le.fit_transform(train_x['Stay'])

In [9]:
X_train = train_x.drop(['Stay', 'case_id', 'patientid'],axis=1)
y_train = train_x['Stay']
X_test =  test_x.drop(['case_id', 'patientid'],axis=1)

In [10]:
X_train.head()
X_train['City_Code_Hospital'].fillna('40', inplace = True)
X_test['City_Code_Hospital'].fillna('40', inplace = True)
X_train['Bed Grade'].fillna('5', inplace = True)
X_test['Bed Grade'].fillna('5', inplace = True)

# Modeling

In [11]:
#Class imbalance
#from sklearn.utils import class_weight
#cw = list(class_weight.compute_class_weight('balanced',
#                                             np.unique(y_train),
#                                            y_train))

In [18]:
# Mention Categorical Values for catboost Model to Handle :
categorical_features = cols
X_data, x_test_data, y_data, y_test_data = train_test_split(X_train, y_train, test_size = 0.3)
#Increased the number of iterations to improve the accuracy
model = CatBoostClassifier(iterations=50, )
model.fit(X_data, y_data, cols)



Learning rate set to 0.5
0:	learn: 1.8583951	total: 4.17s	remaining: 3m 24s
1:	learn: 1.7531697	total: 7.86s	remaining: 3m 8s
2:	learn: 1.6636860	total: 11.7s	remaining: 3m 3s
3:	learn: 1.6206351	total: 15.6s	remaining: 2m 59s
4:	learn: 1.6049457	total: 19.6s	remaining: 2m 56s
5:	learn: 1.5841984	total: 23.6s	remaining: 2m 52s
6:	learn: 1.5739690	total: 27.4s	remaining: 2m 48s
7:	learn: 1.5663220	total: 31.2s	remaining: 2m 43s
8:	learn: 1.5600948	total: 35s	remaining: 2m 39s
9:	learn: 1.5546858	total: 39s	remaining: 2m 35s
10:	learn: 1.5507206	total: 42.8s	remaining: 2m 31s
11:	learn: 1.5466889	total: 46.7s	remaining: 2m 27s
12:	learn: 1.5449998	total: 50.5s	remaining: 2m 23s
13:	learn: 1.5420040	total: 54.4s	remaining: 2m 19s
14:	learn: 1.5395443	total: 58.3s	remaining: 2m 15s
15:	learn: 1.5353442	total: 1m 2s	remaining: 2m 12s
16:	learn: 1.5337774	total: 1m 6s	remaining: 2m 8s
17:	learn: 1.5332758	total: 1m 10s	remaining: 2m 4s
18:	learn: 1.5319328	total: 1m 14s	remaining: 2m 1s
19:	

In [19]:
from sklearn.metrics import classification_report
Y_val_pred = model.predict(x_test_data)
target = ['class1', 'class2','class3','class4','class5','class6','class7','class8', 'class9', 'class10','class11' ]
print (classification_report(y_test_data, Y_val_pred, target_names=target))

              precision    recall  f1-score   support

      class1       0.41      0.14      0.21      7095
      class2       0.43      0.50      0.46     23430
      class3       0.42      0.66      0.51     26103
      class4       0.41      0.22      0.29     16654
      class5       0.15      0.00      0.00      3450
      class6       0.40      0.50      0.45     10503
      class7       0.00      0.00      0.00       841
      class8       0.35      0.01      0.03      3155
      class9       0.34      0.17      0.23      1447
     class10       0.29      0.00      0.00       851
     class11       0.51      0.42      0.46      2003

    accuracy                           0.42     95532
   macro avg       0.34      0.24      0.24     95532
weighted avg       0.40      0.42      0.38     95532



In [20]:
model.fit(X_train, y_train, cols)
Y = model.predict(X_test)

Learning rate set to 0.5
0:	learn: 1.8480776	total: 5.29s	remaining: 4m 19s
1:	learn: 1.7516937	total: 11.3s	remaining: 4m 30s
2:	learn: 1.6893851	total: 18s	remaining: 4m 42s
3:	learn: 1.6344438	total: 25.1s	remaining: 4m 48s
4:	learn: 1.6026584	total: 32.7s	remaining: 4m 54s
5:	learn: 1.5854128	total: 40.3s	remaining: 4m 55s
6:	learn: 1.5762173	total: 48.1s	remaining: 4m 55s
7:	learn: 1.5714055	total: 56.1s	remaining: 4m 54s
8:	learn: 1.5653971	total: 1m 3s	remaining: 4m 51s
9:	learn: 1.5594057	total: 1m 13s	remaining: 4m 53s
10:	learn: 1.5577521	total: 1m 21s	remaining: 4m 49s
11:	learn: 1.5530065	total: 1m 30s	remaining: 4m 45s
12:	learn: 1.5493037	total: 1m 38s	remaining: 4m 39s
13:	learn: 1.5469488	total: 1m 46s	remaining: 4m 33s
14:	learn: 1.5443978	total: 1m 56s	remaining: 4m 30s
15:	learn: 1.5404560	total: 2m 5s	remaining: 4m 26s
16:	learn: 1.5391559	total: 2m 14s	remaining: 4m 20s
17:	learn: 1.5329025	total: 2m 23s	remaining: 4m 14s
18:	learn: 1.5307660	total: 2m 33s	remainin

In [21]:
Y= Y.flatten()

In [22]:
test_submission = pd.read_csv("test.csv")
solution = pd.DataFrame({'case_id':test_submission.case_id, 'Stay':Y}, columns = ['case_id','Stay'])
solution['Stay']=solution['Stay'].map({0:'0-10',1:'11-20',2:'21-30',
                             3:'31-40',4:'41-50',5:'51-60',6:'61-70',7:'71-80',8:'81-90',9:'91-100',10:'More than 100 days'})
solution.to_csv('catboostStay_sol_best.csv', index=False)